<a href="https://colab.research.google.com/github/setigogoli/ML-project/blob/main/customer_churn_prediction%20(5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<center> <span style="color:#0F52BA;font-family:serif; font-size:34px;">
ML Project\
Setayesh Heydari 40104073\
Amir Hossein Shahrabi 401104208\
Amir Abbas Donyadideh 401104113
</span> </center>**

# Loading libraries and Dataset

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "WA_Fn-UseC_-Telco-Customer-Churn.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "blastchar/telco-customer-churn",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

# **Phase 1**

# Understanding the Data(Data Shape)

Each row represents a customer, each column contains customer’s attributes described on the column Metadata.

In [ ]:
df.head()

**The data set includes information about:**
* **Customers who left within the last month** – the column is called Churn

* **Services that each customer has signed up for** – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies

* **Customer account information** - how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges

* **Demographic info about customers** – gender, age range, and if they have partners and dependents

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.columns.values

In [ ]:
df.dtypes


* The target the we will use to guide the exploration is **Churn**

# Find duplicate rows



In [ ]:
# ===== Find duplicate rows =====

duplicate_rows = df.duplicated()

print("Number of duplicate rows:", duplicate_rows.sum())

# See the duplicate rows (optional)
df[duplicate_rows]


In [ ]:
# Remove duplicates (keep first occurrence)
df = df.drop_duplicates()

print("Shape after removing duplicate rows:", df.shape)


# Find & remove duplicate variables (columns)


In [ ]:
# ===== Find & remove duplicate variables (columns) =====

# 1) Duplicate columns by *name* (exact same column label repeated)
dup_name_mask = df.columns.duplicated()
dup_name_cols = df.columns[dup_name_mask].tolist()

if dup_name_cols:
    print("Duplicate column names found:", dup_name_cols)
    # Keep first occurrence, drop the rest
    df = df.loc[:, ~dup_name_mask]
else:
    print("No duplicate column names found.")

# 2) Duplicate columns by *content* (same values in every row)
# Transpose to compare columns as rows, then find duplicates
dup_content_mask = df.T.duplicated()
dup_content_cols = df.columns[dup_content_mask].tolist()

if dup_content_cols:
    print("Duplicate columns by content found:", dup_content_cols)
    df = df.drop(columns=dup_content_cols)
else:
    print("No duplicate columns by content found.")

print("Shape after removing duplicates:", df.shape)


# Histogram using Matplotlib

In [ ]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

plt.figure(figsize=(15,10))

for i, col in enumerate(numeric_cols, 1):
    plt.subplot(2, 2, i)
    sns.histplot(df[col], bins=30, kde=True)
    plt.title(f'Distribution of {col}')

plt.tight_layout()
plt.show()


#Histogram using seaborn

In [ ]:
# Select numeric columns automatically
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Plot histograms
df[numeric_cols].hist(figsize=(12, 8), bins=30)

plt.tight_layout()
plt.show()


# histogeram other way


In [ ]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

for col in numeric_cols:
    fig = px.histogram(df, x=col, nbins=30, title=f'Distribution of {col}')
    fig.show()


# bar plot


In [ ]:
# Select categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Remove customerID (not useful for plotting)
categorical_cols = categorical_cols.drop('customerID')

categorical_cols


In [ ]:
plt.figure(figsize=(18, 25))

for i, col in enumerate(categorical_cols, 1):
    plt.subplot(len(categorical_cols)//2 + 1, 2, i)
    sns.countplot(x=col, data=df)
    plt.title(f'Distribution of {col}')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

# Define the models and their predictions
models = {
    "Logistic Regression": lr_model.predict(X_test),
    "KNN": knn_model.predict(X_test),
    "Random Forest": model_rf.predict(X_test),
    "Gradient Boosting": gb.predict(X_test)
}

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for i, (name, y_pred) in enumerate(models.items()):
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Churn', 'Churn'])
    disp.plot(cmap='Blues', ax=axes[i], colorbar=False)
    axes[i].set_title(f'{name} Confusion Matrix', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Predicted Label', fontsize=12)
    axes[i].set_ylabel('True Label', fontsize=12)

plt.suptitle('Comparison of Confusion Matrices', fontsize=18, fontweight='bold', y=1.02)
plt.tight_layout(rect=[0, 0, 1, 0.98]) # Adjust layout to prevent suptitle overlap
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

# Define the models and their predictions
models = {
    "Logistic Regression": lr_model.predict(X_test),
    "KNN": knn_model.predict(X_test),
    "Random Forest": model_rf.predict(X_test),
    "Gradient Boosting": gb.predict(X_test)
}

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for i, (name, y_pred) in enumerate(models.items()):
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Churn', 'Churn'])
    disp.plot(cmap='Blues', ax=axes[i], colorbar=False)
    axes[i].set_title(f'{name} Confusion Matrix', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Predicted Label', fontsize=12)
    axes[i].set_ylabel('True Label', fontsize=12)

plt.suptitle('Comparison of Confusion Matrices', fontsize=18, fontweight='bold', y=1.02)
plt.tight_layout(rect=[0, 0, 1, 0.98]) # Adjust layout to prevent suptitle overlap
plt.show()

# Bar plot other way

In [ ]:
plt.figure(figsize=(18, 25))

for i, col in enumerate(categorical_cols, 1):
    plt.subplot(len(categorical_cols)//2 + 1, 2, i)

    df[col].value_counts().plot(kind='bar')

    plt.title(f'Distribution of {col}')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
for col in categorical_cols:
    fig = px.histogram(df, x=col, title=f'Distribution of {col}')
    fig.show()


<a id = "7" ></a>
# <span style="font-family:serif; font-size:28px;"> 4. Visualize missing values </span>
<a id = "missingvalue" ></a>

In [ ]:
# Visualize missing values as a matrix
msno.matrix(df);

> Using this matrix we can very quickly find the pattern of missingness in the dataset.
* From the above visualisation we can observe that it has no peculiar pattern that stands out. In fact there is no missing data.

***

# Data Manipulation(finding missing values)

In [ ]:
df = df.drop(['customerID'], axis = 1)
df.head()

* On deep analysis, we can find some indirect missingness in our data (which can be in form of blankspaces). Let's see that!

In [ ]:
df['TotalCharges'] = pd.to_numeric(df.TotalCharges, errors='coerce')
df.isnull().sum()

* Here we see that the TotalCharges has 11 missing values. Let's check this data.

In [ ]:
df[np.isnan(df['TotalCharges'])]

* It can also be noted that the Tenure column is 0 for these entries even though the MonthlyCharges column is not empty.

Let's see if there are any other 0 values in the tenure column.

In [ ]:
df[df['tenure'] == 0].index

* There are no additional missing values in the Tenure column.

Let's delete the rows with missing values in Tenure columns since there are only 11 rows and deleting them will not affect the data.

In [ ]:
df.drop(labels=df[df['tenure'] == 0].index, axis=0, inplace=True)
df[df['tenure'] == 0].index

> To solve the problem of missing values in TotalCharges column, I decided to fill it with the mean of TotalCharges values.

In [ ]:
df.fillna(df["TotalCharges"].mean())

In [ ]:
df.isnull().sum()

In [ ]:
df["SeniorCitizen"]= df["SeniorCitizen"].map({0: "No", 1: "Yes"})
df.head()

In [ ]:
df["InternetService"].describe(include=['object', 'bool'])

In [ ]:
numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
df[numerical_cols].describe()

In [ ]:
print(df.isnull().sum())

#distribution analysis

In [ ]:
g_labels = ['Male', 'Female']
c_labels = ['No', 'Yes']
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=g_labels, values=df['gender'].value_counts(), name="Gender"),
              1, 1)
fig.add_trace(go.Pie(labels=c_labels, values=df['Churn'].value_counts(), name="Churn"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name", textfont_size=16)

fig.update_layout(
    title_text="Gender and Churn Distributions",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Gender', x=0.16, y=0.5, font_size=20, showarrow=False),
                 dict(text='Churn', x=0.84, y=0.5, font_size=20, showarrow=False)])
fig.show()

* 26.6 % of customers switched to another firm.
* Customers are 49.5 % female and 50.5 % male.

In [ ]:
df["Churn"][df["Churn"]=="No"].groupby(by=df["gender"]).count()

In [ ]:
df["Churn"][df["Churn"]=="Yes"].groupby(by=df["gender"]).count()

In [ ]:
plt.figure(figsize=(6, 6))
labels =["Churn: Yes","Churn:No"]
values = [1869,5163]
labels_gender = ["F","M","F","M"]
sizes_gender = [939,930 , 2544,2619]
colors = ['#ff6666', '#66b3ff']
colors_gender = ['#c2c2f0','#ffb3e6', '#c2c2f0','#ffb3e6']
explode = (0.3,0.3)
explode_gender = (0.1,0.1,0.1,0.1)
textprops = {"fontsize":15}
#Plot
plt.pie(values, labels=labels,autopct='%1.1f%%',pctdistance=1.08, labeldistance=0.8,colors=colors, startangle=90,frame=True, explode=explode,radius=10, textprops =textprops, counterclock = True, )
plt.pie(sizes_gender,labels=labels_gender,colors=colors_gender,startangle=90, explode=explode_gender,radius=7, textprops =textprops, counterclock = True, )
#Draw circle
centre_circle = plt.Circle((0,0),5,color='black', fc='white',linewidth=0)
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.title('Churn Distribution w.r.t Gender: Male(M), Female(F)', fontsize=15, y=1.1)

# show plot

plt.axis('equal')
plt.tight_layout()
plt.show()

* There is negligible difference in customer percentage/ count who chnaged the service provider. Both genders behaved in similar fashion when it comes to migrating to another service provider/firm.

In [ ]:
fig = px.histogram(df, x="Churn", color="Contract", barmode="group", title="<b>Customer contract distribution<b>")
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

* About 75% of customer with Month-to-Month Contract opted to move out as compared to 13% of customrs with One Year Contract and 3% with Two Year Contract

In [ ]:
labels = df['PaymentMethod'].unique()
values = df['PaymentMethod'].value_counts()

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.update_layout(title_text="<b>Payment Method Distribution</b>")
fig.show()

In [ ]:
fig = px.histogram(df, x="Churn", color="PaymentMethod", title="<b>Customer Payment Method distribution w.r.t. Churn</b>")
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

* Major customers who moved out were having Electronic Check as Payment Method.
* Customers who opted for Credit-Card automatic transfer or Bank Automatic Transfer and Mailed Check as Payment Method were less likely to move out.  

In [ ]:
df["InternetService"].unique()

In [ ]:
df[df["gender"]=="Male"][["InternetService", "Churn"]].value_counts()

In [ ]:
df[df["gender"]=="Female"][["InternetService", "Churn"]].value_counts()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
  x = [['Churn:No', 'Churn:No', 'Churn:Yes', 'Churn:Yes'],
       ["Female", "Male", "Female", "Male"]],
  y = [965, 992, 219, 240],
  name = 'DSL',
))

fig.add_trace(go.Bar(
  x = [['Churn:No', 'Churn:No', 'Churn:Yes', 'Churn:Yes'],
       ["Female", "Male", "Female", "Male"]],
  y = [889, 910, 664, 633],
  name = 'Fiber optic',
))

fig.add_trace(go.Bar(
  x = [['Churn:No', 'Churn:No', 'Churn:Yes', 'Churn:Yes'],
       ["Female", "Male", "Female", "Male"]],
  y = [690, 717, 56, 57],
  name = 'No Internet',
))

fig.update_layout(title_text="<b>Churn Distribution w.r.t. Internet Service and Gender</b>")

fig.show()

* A lot of customers choose the Fiber optic service and it's also evident that the customers who use Fiber optic have high churn rate, this might suggest a dissatisfaction with this type of internet service.
* Customers having DSL service are majority in number and have less churn rate compared to Fibre optic service.

In [ ]:
color_map = {"Yes": "#FF97FF", "No": "#AB63FA"}
fig = px.histogram(df, x="Churn", color="Dependents", barmode="group", title="<b>Dependents distribution</b>", color_discrete_map=color_map)
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

* Customers without dependents are more likely to churn

In [ ]:
color_map = {"Yes": '#FFA15A', "No": '#00CC96'}
fig = px.histogram(df, x="Churn", color="Partner", barmode="group", title="<b>Chrun distribution w.r.t. Partners</b>", color_discrete_map=color_map)
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

* Customers that doesn't have partners are more likely to churn

In [ ]:
color_map = {"Yes": '#00CC96', "No": '#B6E880'}
fig = px.histogram(df, x="Churn", color="SeniorCitizen", title="<b>Chrun distribution w.r.t. Senior Citizen</b>", color_discrete_map=color_map)
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

* It can be observed that the fraction of senior citizen is very less.
* Most of the senior citizens churn.

In [ ]:
color_map = {"Yes": "#FF97FF", "No": "#AB63FA"}
fig = px.histogram(df, x="Churn", color="OnlineSecurity", barmode="group", title="<b>Churn w.r.t Online Security</b>", color_discrete_map=color_map)
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

* Most customers churn in the absence of online security,

In [ ]:
color_map = {"Yes": '#FFA15A', "No": '#00CC96'}
fig = px.histogram(df, x="Churn", color="PaperlessBilling",  title="<b>Chrun distribution w.r.t. Paperless Billing</b>", color_discrete_map=color_map)
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

* Customers with Paperless Billing are most likely to churn.

In [ ]:
fig = px.histogram(df, x="Churn", color="TechSupport",barmode="group",  title="<b>Chrun distribution w.r.t. TechSupport</b>")
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

* Customers with no TechSupport are most likely to migrate to another service provider.

In [ ]:
color_map = {"Yes": '#00CC96', "No": '#B6E880'}
fig = px.histogram(df, x="Churn", color="PhoneService", title="<b>Chrun distribution w.r.t. Phone Service</b>", color_discrete_map=color_map)
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

* Very small fraction of customers don't have a phone service and out of that, 1/3rd Customers are more likely to churn.

In [ ]:
sns.set_context("paper",font_scale=1.1)
ax = sns.kdeplot(df.MonthlyCharges[(df["Churn"] == 'No') ],
                color="Red", shade = True);
ax = sns.kdeplot(df.MonthlyCharges[(df["Churn"] == 'Yes') ],
                ax =ax, color="Blue", shade= True);
ax.legend(["Not Churn","Churn"],loc='upper right');
ax.set_ylabel('Density');
ax.set_xlabel('Monthly Charges');
ax.set_title('Distribution of monthly charges by churn');


* Customers with higher Monthly Charges are also more likely to churn

In [ ]:
ax = sns.kdeplot(df.TotalCharges[(df["Churn"] == 'No') ],
                color="Gold", shade = True);
ax = sns.kdeplot(df.TotalCharges[(df["Churn"] == 'Yes') ],
                ax =ax, color="Green", shade= True);
ax.legend(["Not Chu0rn","Churn"],loc='upper right');
ax.set_ylabel('Density');
ax.set_xlabel('Total Charges');
ax.set_title('Distribution of total charges by churn');

#Heat Map

In [ ]:
plt.figure(figsize=(25, 10))

corr = df.apply(lambda x: pd.factorize(x)[0]).corr()

# Create a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

ax = sns.heatmap(corr, mask=mask, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, linewidths=.2, cmap='coolwarm', vmin=-1, vmax=1)

___

# Box Plot


Tenure vs Churn


In [ ]:
fig = px.box(df, x='Churn', y='tenure', color='Churn')

fig.update_yaxes(title_text='Tenure (Months)')
fig.update_xaxes(title_text='Churn')

fig.update_layout(
    autosize=True,
    width=750,
    height=600,
    title_font=dict(size=25, family='Courier'),
    title='<b>Tenure vs Churn</b>'
)

fig.show()


MonthlyCharges vs Churn

In [ ]:
fig = px.box(df, x='Churn', y='MonthlyCharges', color='Churn')

fig.update_yaxes(title_text='Monthly Charges')
fig.update_xaxes(title_text='Churn')

fig.update_layout(
    autosize=True,
    width=750,
    height=600,
    title_font=dict(size=25, family='Courier'),
    title='<b>Monthly Charges vs Churn</b>'
)

fig.show()


TotalCharges vs Churn

In [ ]:
#Make sure it's numeric first
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
df = df.dropna(subset=["TotalCharges"])

fig = px.box(df, x='Churn', y='TotalCharges', color='Churn')

fig.update_yaxes(title_text='Total Charges')
fig.update_xaxes(title_text='Churn')

fig.update_layout(
    autosize=True,
    width=750,
    height=600,
    title_font=dict(size=25, family='Courier'),
    title='<b>Total Charges vs Churn</b>'
)

fig.show()



all in one code

In [ ]:
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

for col in numeric_cols:
    fig = px.box(df, x='Churn', y=col, color='Churn',
                 title=f'<b>{col} vs Churn</b>')

    fig.update_layout(width=750, height=600)
    fig.show()


* New customers are more likely to churn

# **Phase 2**

#Implement Imputation


In [ ]:

# Convert TotalCharges to numeric (blank strings -> NaN)
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

# Separate numeric and categorical columns
num_cols = df.select_dtypes(include=["int64", "float64"]).columns
cat_cols = df.select_dtypes(include=["object", "category", "bool"]).columns


In [ ]:
# Numeric: fill NaN with median (robust)
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# Categorical: fill NaN with most frequent value (mode)
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# Check
print("Missing values after imputation:\n", df.isnull().sum()[df.isnull().sum() > 0])


another way

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Convert TotalCharges to numeric
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

# Split X/y
X = df.drop(columns=["Churn"])
y = df["Churn"]

# Identify columns
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object", "category", "bool"]).columns

# Preprocessors
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ]
)

# Example split (optional)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Fit/transform
X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared  = preprocessor.transform(X_test)

print("Done. Shapes:", X_train_prepared.shape, X_test_prepared.shape)


#Lable Encoding

In [ ]:
# Select categorical columns
cat_cols = df.select_dtypes(include=['object']).columns

# Find binary columns (exactly 2 unique values)
binary_cols = [col for col in cat_cols if df[col].nunique() == 2]

print("Binary columns:")
print(binary_cols)


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in binary_cols:
    df[col] = le.fit_transform(df[col])

df.head()


In [ ]:
df[binary_cols].head()

#One Hot Encoding

In [ ]:
# Select categorical columns
cat_cols = df.select_dtypes(include=['object']).columns

# Select columns with more than 2 unique values
multi_cols = [col for col in cat_cols if df[col].nunique() > 2]

print("Multi-category columns:")
print(multi_cols)


In [ ]:
df = pd.get_dummies(df, columns=multi_cols, drop_first=True)

df.head()
print("New shape after One-Hot Encoding:", df.shape)


another way

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# 1) Recompute multi-category columns from CURRENT df (only object columns with >2 unique values)
multi_cols = [c for c in df.columns if df[c].dtype == "object" and df[c].nunique() > 2]

print("Multi-category columns to one-hot encode:", multi_cols)

# If nothing left to encode, stop safely
if len(multi_cols) == 0:
    print("No multi-category object columns left to encode.")
else:
    # 2) One-hot encode
    ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)
    encoded = ohe.fit_transform(df[multi_cols])

    encoded_df = pd.DataFrame(
        encoded,
        columns=ohe.get_feature_names_out(multi_cols),
        index=df.index
    )

    # 3) Replace original columns with encoded columns
    df = df.drop(columns=multi_cols)
    df = pd.concat([df, encoded_df], axis=1)

    print("Done. New shape:", df.shape)


In [ ]:
multi_cols = [c for c in df.columns if df[c].dtype == "object" and df[c].nunique() >= 2]


## train test split

In [ ]:
def object_to_int(dataframe_series):
    if dataframe_series.dtype=='object':
        dataframe_series = LabelEncoder().fit_transform(dataframe_series)
    return dataframe_series

In [ ]:
df = df.apply(lambda x: object_to_int(x))
df.head()

In [ ]:
plt.figure(figsize=(14,7))
df.corr()['Churn'].sort_values(ascending = False)

In [ ]:
X = df.drop(columns = ['Churn'])
y = df['Churn'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.30, random_state = 40, stratify=y)

In [ ]:
def distplot(feature, frame, color='r'):
    plt.figure(figsize=(8,3))
    plt.title("Distribution for {}".format(feature))
    ax = sns.distplot(frame[feature], color= color)

In [ ]:
num_cols = ["tenure", 'MonthlyCharges', 'TotalCharges']
for feat in num_cols: distplot(feat, df)

Since the numerical features are distributed over different value ranges, I will use standard scalar to scale them down to the same range.

#Standard scaler

In [ ]:
df_std = pd.DataFrame(StandardScaler().fit_transform(df[num_cols].astype('float64')),
                       columns=num_cols)
for feat in numerical_cols: distplot(feat, df_std, color='c')

In [ ]:
# Divide the columns into 3 categories, one ofor standardisation, one for label encoding and one for one hot encoding

cat_cols_ohe =['PaymentMethod', 'Contract', 'InternetService'] # those that need one-hot encoding
cat_cols_le = list(set(X_train.columns)- set(num_cols) - set(cat_cols_ohe)) #those that need label encoding

In [ ]:
scaler= StandardScaler()

X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

# **Phase 3**

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

# ── make sure boolean columns are int (required for chi2) ──
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

## Feature Engineering — Creating 2 new meaningful features


In [ ]:
# ── Feature 1: Tenure Group (بازه‌بندی tenure) ──────────────
# Binning tenure into meaningful loyalty segments
def tenure_group(tenure):
    if tenure <= 12:
        return 0   # New Customer (0–1 year)
    elif tenure <= 24:
        return 1   # Developing (1–2 years)
    elif tenure <= 48:
        return 2   # Established (2–4 years)
    else:
        return 3   # Loyal (4+ years)

df['Tenure_Group'] = df['tenure'].apply(tenure_group)

print("Tenure Group distribution:")
print(df['Tenure_Group'].value_counts().sort_index())
print()

# ── Feature 2: Charges per Month Ratio ──────────────────────
# TotalCharges / tenure gives average spend per month
# This reveals if a customer's spending is consistent or changed
df['Avg_Monthly_Spend'] = df['TotalCharges'] / (df['tenure'] + 1)
# (+1 to avoid division by zero for tenure=0)

# ── Feature 3: Service Count ────────────────────────────────
# How many add-on services does each customer subscribe to?
# More services → higher switching cost → lower churn probability
service_cols = [
    'MultipleLines_Yes',
    'OnlineSecurity_Yes',
    'OnlineBackup_Yes',
    'DeviceProtection_Yes',
    'TechSupport_Yes',
    'StreamingTV_Yes',
    'StreamingMovies_Yes'
]
df['Service_Count'] = df[service_cols].sum(axis=1)

print("New features added:")
print(df[['tenure', 'Tenure_Group', 'TotalCharges',
          'Avg_Monthly_Spend', 'Service_Count']].head(10))
print()
print("Correlation of new features with Churn:")
print(df[['Tenure_Group', 'Avg_Monthly_Spend',
          'Service_Count', 'Churn']].corr()['Churn'])


## Filter-Based Selection (Chi-Squared + ANOVA)

In [ ]:
# Separate features and target
X = df.drop(columns=['Churn'])
y = df['Churn']

# ── Scale numerical features to [0,1] for Chi2 ──────────────
# Chi2 requires non-negative values
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(X),
    columns=X.columns
)


## Chi-Squared Test (for categorical/binary features)

In [ ]:
chi2_selector = SelectKBest(score_func=chi2, k='all')
chi2_selector.fit(X_scaled, y)

chi2_scores = pd.DataFrame({
    'Feature': X.columns,
    'Chi2_Score': chi2_selector.scores_,
    'P_Value': chi2_selector.pvalues_
}).sort_values('Chi2_Score', ascending=False).reset_index(drop=True)

print("=" * 55)
print("Chi-Squared Scores (Top 15):")
print("=" * 55)
print(chi2_scores.head(15).to_string(index=False))
print()

# Plot Chi2
plt.figure(figsize=(12, 7))
sns.barplot(
    data=chi2_scores.head(15),
    x='Chi2_Score',
    y='Feature',
    palette='Blues_r'
)
plt.title('Top 15 Features — Chi-Squared Test', fontsize=14, fontweight='bold')
plt.xlabel('Chi² Score')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

## ANOVA F-Test (better for continuous numerical features)

In [ ]:
anova_selector = SelectKBest(score_func=f_classif, k='all')
anova_selector.fit(X_scaled, y)

anova_scores = pd.DataFrame({
    'Feature': X.columns,
    'ANOVA_F_Score': anova_selector.scores_,
    'P_Value': anova_selector.pvalues_
}).sort_values('ANOVA_F_Score', ascending=False).reset_index(drop=True)

print("=" * 55)
print("ANOVA F-Scores (Top 15):")
print("=" * 55)
print(anova_scores.head(15).to_string(index=False))
print()

# Plot ANOVA
plt.figure(figsize=(12, 7))
sns.barplot(
    data=anova_scores.head(15),
    x='ANOVA_F_Score',
    y='Feature',
    palette='Greens_r'
)
plt.title('Top 15 Features — ANOVA F-Test', fontsize=14, fontweight='bold')
plt.xlabel('ANOVA F-Score')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

## Combined Filter Score

In [ ]:
filter_combined = chi2_scores[['Feature', 'Chi2_Score']].merge(
    anova_scores[['Feature', 'ANOVA_F_Score']], on='Feature'
)
# Normalize both scores to [0,1] and average them
filter_combined['Chi2_norm']  = (filter_combined['Chi2_Score'] /
                                  filter_combined['Chi2_Score'].max())
filter_combined['ANOVA_norm'] = (filter_combined['ANOVA_F_Score'] /
                                  filter_combined['ANOVA_F_Score'].max())
filter_combined['Combined_Score'] = (filter_combined['Chi2_norm'] +
                                      filter_combined['ANOVA_norm']) / 2
filter_combined = filter_combined.sort_values(
    'Combined_Score', ascending=False
).reset_index(drop=True)

top15_filter = filter_combined.head(15)['Feature'].tolist()
print("Top 15 Features by Combined Filter Score:")
print(top15_filter)

## Lasso Regression (L1 — drives irrelevant features to 0)

In [ ]:
# 1. Redefine X and y using df_model (the feature-selected DataFrame)
X = df_model.drop(columns=['Churn'])
y = df_model['Churn']

# 2. Re-perform train-test split using this X and y
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=40, stratify=y
)

# 3. Identify numerical columns within the selected features for scaling
numerical_cols_for_scaling_selected = ['tenure', 'Avg_Monthly_Spend', 'TotalCharges', 'MonthlyCharges']
current_numerical_cols = [col for col in numerical_cols_for_scaling_selected if col in X_train.columns]

# 4. Apply StandardScaler to the numerical columns of the newly split X_train and X_test
scaler = StandardScaler()
if current_numerical_cols:
    X_train[current_numerical_cols] = scaler.fit_transform(X_train[current_numerical_cols])
    X_test[current_numerical_cols] = scaler.transform(X_test[current_numerical_cols])

print("X_train and X_test have been updated to use final selected features and scaled.")
print(f"New X_train shape: {X_train.shape}")
print(f"New X_test shape: {X_test.shape}")

## Random Forest Feature Importance

In [ ]:
rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)
rf.fit(X_train, y_train)

rf_importance = pd.DataFrame({
    'Feature': X.columns,
    'RF_Importance': rf.feature_importances_
}).sort_values('RF_Importance', ascending=False).reset_index(drop=True)

print("=" * 55)
print("Random Forest — Top 15 Feature Importances:")
print("=" * 55)
print(rf_importance.head(15).to_string(index=False))
print()

# Plot RF
plt.figure(figsize=(12, 7))
sns.barplot(
    data=rf_importance.head(15),
    x='RF_Importance',
    y='Feature',
    palette='Purples_r'
)
plt.title('Feature Importance — Random Forest', fontsize=14, fontweight='bold')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

## Final Feature Subset — Combining all methods

In [ ]:
# Rank features across all 3 methods
all_features = X.columns.tolist()

# Rank by each method (lower rank = more important)
filter_rank = {f: i for i, f in enumerate(top15_filter)}
lasso_rank  = {f: i for i, f in
               enumerate(lasso_importance['Feature'].tolist())}
rf_rank     = {f: i for i, f in
               enumerate(rf_importance['Feature'].tolist())}

ranking_df = pd.DataFrame({'Feature': all_features})
ranking_df['Filter_Rank'] = ranking_df['Feature'].map(
    lambda f: filter_rank.get(f, len(all_features))
)
ranking_df['Lasso_Rank']  = ranking_df['Feature'].map(
    lambda f: lasso_rank.get(f, len(all_features))
)
ranking_df['RF_Rank']     = ranking_df['Feature'].map(
    lambda f: rf_rank.get(f, len(all_features))
)
ranking_df['Avg_Rank']    = ranking_df[
    ['Filter_Rank', 'Lasso_Rank', 'RF_Rank']
].mean(axis=1)

ranking_df = ranking_df.sort_values('Avg_Rank').reset_index(drop=True)

print("=" * 60)
print("Final Feature Ranking (All Methods Combined):")
print("=" * 60)
print(ranking_df.head(15).to_string(index=False))

# ── Select Top 12 Final Features ─────────────────────────────
final_features = ranking_df.head(12)['Feature'].tolist()
print("\n✅ Final Selected Features:")
for i, f in enumerate(final_features, 1):
    print(f"  {i:2}. {f}")

# ── Final dataframe ready for modeling ───────────────────────
df_model = df[final_features + ['Churn']].copy()
print(f"\ndf_model shape: {df_model.shape}")
print(df_model.head())


In [ ]:
# ============================================================
# E. Textual Justification for Final Feature Subset
# ============================================================

justification = """
╔══════════════════════════════════════════════════════════════╗
║         PHASE 3 — FINAL FEATURE SUBSET JUSTIFICATION        ║
╚══════════════════════════════════════════════════════════════╝

1. FEATURE ENGINEERING
   ─────────────────────────────────────────────────────────
   • Tenure_Group: Binning tenure into 4 loyalty segments
     (New/Developing/Established/Loyal) captures non-linear
     churn behavior — new customers churn at much higher rates.

   • Avg_Monthly_Spend: TotalCharges / (tenure+1) captures
     whether a customer's spending is rising or falling over
     time, which is more informative than raw TotalCharges alone.

   • Service_Count: The total number of add-on services acts
     as a proxy for switching cost — customers with more
     services face higher friction when leaving.

2. FILTER-BASED SELECTION (Chi2 + ANOVA)
   ─────────────────────────────────────────────────────────
   • Chi-Squared identified categorical features most
     statistically dependent on Churn (p < 0.05).
   • ANOVA F-Test confirmed continuous features (tenure,
     MonthlyCharges, TotalCharges) with highest group
     mean differences between churned/non-churned customers.
   • Features failing both tests (p > 0.05 in both) were
     considered statistically insignificant and down-ranked.

3. MODEL-BASED SELECTION
   ─────────────────────────────────────────────────────────
   • Lasso (L1): By penalizing coefficients toward zero,
     Lasso automatically eliminated multicollinear and
     redundant features. Only features surviving L1
     shrinkage carry independent predictive signal.
   • Random Forest: Impurity-based importance scores
     capture non-linear relationships and interactions
     that linear methods like Lasso may miss.

4. FINAL SELECTION RATIONALE
   ─────────────────────────────────────────────────────────
   The final 12 features were chosen by averaging ranks
   across all three methods. This ensemble approach is more
   robust than relying on any single method:
   - It avoids overfitting to one selection criterion.
   - Features consistently ranked high across methods
     are genuinely predictive, not method-specific artifacts.
   - Multicollinear OHE dummy pairs (e.g.,
     'InternetService_No' vs 'InternetService_Fiber optic')
     were deduplicated keeping only the higher-ranked one.
"""
print(justification)


# Phase 4

## Check Class Distribution

In [ ]:
from collections import Counter

# ── Current distribution ─────────────────────────────────────
print("=" * 55)
print("Class Distribution in Full Dataset:")
print("=" * 55)
print(f"  Not Churned (0): {Counter(y)[0]:,}  ({Counter(y)[0]/len(y)*100:.1f}%)")
print(f"  Churned     (1): {Counter(y)[1]:,}  ({Counter(y)[1]/len(y)*100:.1f}%)")
print(f"  Imbalance Ratio: {Counter(y)[0]/Counter(y)[1]:.2f} : 1")

# ── Visualize ────────────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Bar chart
counts = pd.Series(y).value_counts().sort_index()
axes[0].bar(['Not Churned (0)', 'Churned (1)'],
            counts.values,
            color=['#2196F3', '#F44336'],
            edgecolor='black', width=0.5)
axes[0].set_title('Class Distribution — Before Balancing',
                   fontsize=13, fontweight='bold')
axes[0].set_ylabel('Count')
for i, v in enumerate(counts.values):
    axes[0].text(i, v + 50, f'{v:,}\n({v/len(y)*100:.1f}%)',
                 ha='center', fontweight='bold')

# Pie chart
axes[1].pie(counts.values,
            labels=['Not Churned', 'Churned'],
            colors=['#2196F3', '#F44336'],
            autopct='%1.1f%%',
            startangle=90,
            explode=(0, 0.05))
axes[1].set_title('Churn Rate', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()


## SMOTE — Synthetic Minority Over-Sampling Technique

In [ ]:
from imblearn.over_sampling import SMOTE

# Use X_train / y_train from Phase 3 (already scaled)
print("Before SMOTE:")
print(f"  Train set — Class 0: {Counter(y_train)[0]:,} | Class 1: {Counter(y_train)[1]:,}")

smote = SMOTE(random_state=42, k_neighbors=5)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("\nAfter SMOTE:")
print(f"  Train set — Class 0: {Counter(y_train_smote)[0]:,} | Class 1: {Counter(y_train_smote)[1]:,}")
print(f"  New training size  : {len(X_train_smote):,} samples")
print(f"  Synthetic samples  : {Counter(y_train_smote)[1] - Counter(y_train)[1]:,} added")

# ── Visualize Before vs After ────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for ax, (counts, title) in zip(axes, [
    (Counter(y_train),       'Training Set — Before SMOTE'),
    (Counter(y_train_smote), 'Training Set — After SMOTE'),
]):
    ax.bar(['Not Churned (0)', 'Churned (1)'],
           [counts[0], counts[1]],
           color=['#2196F3', '#F44336'],
           edgecolor='black', width=0.5)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_ylabel('Count')
    total = counts[0] + counts[1]
    for i, (k, v) in enumerate(sorted(counts.items())):
        ax.text(i, v + 30, f'{v:,}\n({v/total*100:.1f}%)',
                ha='center', fontweight='bold')

plt.tight_layout()
plt.show()


## Compare: SMOTE vs class_weight='balanced'

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (classification_report, roc_auc_score,
                              ConfusionMatrixDisplay, f1_score)

results = {}

# ── C1. Baseline (no balancing) ──────────────────────────────
lr_base = LogisticRegression(max_iter=1000, random_state=42)
lr_base.fit(X_train, y_train)
y_pred_base = lr_base.predict(X_test)
results['Baseline'] = {
    'AUC'     : roc_auc_score(y_test, lr_base.predict_proba(X_test)[:, 1]),
    'F1_churn': f1_score(y_test, y_pred_base),
    'Report'  : classification_report(y_test, y_pred_base)
}

# ── C2. Class Weights ────────────────────────────────────────
lr_cw = LogisticRegression(class_weight='balanced',
                            max_iter=1000, random_state=42)
lr_cw.fit(X_train, y_train)
y_pred_cw = lr_cw.predict(X_test)
results['Class_Weight'] = {
    'AUC'     : roc_auc_score(y_test, lr_cw.predict_proba(X_test)[:, 1]),
    'F1_churn': f1_score(y_test, y_pred_cw),
    'Report'  : classification_report(y_test, y_pred_cw)
}

# ── C3. SMOTE ────────────────────────────────────────────────
lr_smote = LogisticRegression(max_iter=1000, random_state=42)
lr_smote.fit(X_train_smote, y_train_smote)
y_pred_smote = lr_smote.predict(X_test)
results['SMOTE'] = {
    'AUC'     : roc_auc_score(y_test, lr_smote.predict_proba(X_test)[:, 1]),
    'F1_churn': f1_score(y_test, y_pred_smote),
    'Report'  : classification_report(y_test, y_pred_smote)
}

# ── Summary Table ────────────────────────────────────────────
print("=" * 55)
print("Comparison Summary")
print("=" * 55)
summary = pd.DataFrame({
    name: {'ROC-AUC': f"{v['AUC']:.4f}",
           'F1 (Churn)': f"{v['F1_churn']:.4f}"}
    for name, v in results.items()
}).T
print(summary.to_string())
print()

# ── Detailed reports ─────────────────────────────────────────
for name, v in results.items():
    print(f"\n--- {name} ---")
    print(v['Report'])

# ── Confusion Matrices ───────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
for ax, (name, pred) in zip(axes, [
    ('Baseline',     y_pred_base),
    ('Class Weight', y_pred_cw),
    ('SMOTE',        y_pred_smote),
]):
    ConfusionMatrixDisplay.from_predictions(
        y_test, pred,
        display_labels=['Not Churn', 'Churn'],
        colorbar=False, ax=ax,
        cmap='Blues'
    )
    ax.set_title(f'{name}\nAUC={results[list(results.keys())[list(results.keys()).index(name.replace(" ","_")) if name.replace(" ","_") in results else name]][ "AUC"]:.4f}',
                 fontsize=11, fontweight='bold')

plt.suptitle('Confusion Matrix Comparison', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# ── Bar chart: AUC & F1 comparison ───────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
methods = list(results.keys())
aucs  = [results[m]['AUC']      for m in methods]
f1s   = [results[m]['F1_churn'] for m in methods]

for ax, (vals, title, color) in zip(axes, [
    (aucs, 'ROC-AUC Score',   ['#42A5F5', '#66BB6A', '#FFA726']),
    (f1s,  'F1 Score (Churn)',['#42A5F5', '#66BB6A', '#FFA726']),
]):
    bars = ax.bar(methods, vals, color=color, edgecolor='black', width=0.5)
    ax.set_ylim(min(vals) - 0.05, 1.0)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_ylabel('Score')
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x() + bar.get_width()/2,
                bar.get_height() + 0.005,
                f'{v:.4f}', ha='center', fontweight='bold')

plt.suptitle('Balancing Methods — Performance Comparison',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()


#Base models

## Logistic Regression




In [ ]:
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)
accuracy_lr = lr_model.score(X_test,y_test)
print("Logistic Regression accuracy is :",accuracy_lr)

In [ ]:
lr_pred= lr_model.predict(X_test)
report = classification_report(y_test,lr_pred)
print(report)

In [ ]:
plt.figure(figsize=(6, 5)) # Slightly larger figure
cm = confusion_matrix(y_test, lr_pred)
sns.heatmap(cm,
            annot=True,
            fmt="d", # Ensure integer format for counts
            cmap='Blues', # Use a blue colormap
            cbar=True, # Add a color bar
            linewidths=0.5, # Thinner lines
            linecolor='lightgray', # Light gray lines
            xticklabels=['Predicted: No Churn', 'Predicted: Churn'], # Add descriptive x-axis labels
            yticklabels=['True: No Churn', 'True: Churn'], # Add descriptive y-axis labels
            annot_kws={"size": 12} # Adjust annotation font size
           )

plt.title("LOGISTIC REGRESSION CONFUSION MATRIX", fontsize=16, fontweight='bold')
plt.xlabel("Predicted Label", fontsize=12) # Add x-label
plt.ylabel("True Label", fontsize=12) # Add y-label
plt.tight_layout() # Adjust layout to prevent labels overlapping
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred_prob = lr_model.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
auc_score = roc_auc_score(y_test, y_pred_prob)

plt.figure(figsize=(8, 7)) # Increased figure size
plt.plot([0, 1], [0, 1], 'k--', label='Random Chance') # Changed to 'k--' for a dashed line and added label
plt.plot(fpr, tpr, label=f'Logistic Regression (AUC = {auc_score:.2f})', color="#1f77b4", linewidth=2) # Blue color, thicker line, and AUC in label
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title(f'Logistic Regression ROC Curve', fontsize=16, fontweight='bold') # Bolder title
plt.legend(loc='lower right', fontsize=10) # Added legend for clarity
plt.grid(True, linestyle='--', alpha=0.7) # Added a grid
plt.tight_layout()
plt.show()

## KNN

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors = 11)
knn_model.fit(X_train,y_train)
predicted_y = knn_model.predict(X_test)
accuracy_knn = knn_model.score(X_test,y_test)
print("KNN accuracy:",accuracy_knn)

In [ ]:
print(classification_report(y_test, predicted_y))

In [ ]:
# Confusion Matrix
plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_test, predicted_y)
sns.heatmap(cm,
            annot=True,
            fmt="d",
            cmap='Blues',
            cbar=True,
            linewidths=0.5,
            linecolor='lightgray',
            xticklabels=['Predicted: No Churn', 'Predicted: Churn'],
            yticklabels=['True: No Churn', 'True: Churn'],
            annot_kws={"size": 12}
           )
plt.title("KNN CONFUSION MATRIX", fontsize=16, fontweight='bold')
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.tight_layout()
plt.show()

## Random Forest

In [ ]:
model_rf = RandomForestClassifier(n_estimators=500 , oob_score = True, n_jobs = -1,
                                  random_state =50, max_features = "sqrt", # Changed 'auto' to 'sqrt'
                                  max_leaf_nodes = 30)
model_rf.fit(X_train, y_train)

# Make predictions
prediction_test = model_rf.predict(X_test)
print ("Random Forest",metrics.accuracy_score(y_test, prediction_test))

In [ ]:
print(classification_report(y_test, prediction_test))

In [ ]:
# Confusion Matrix
plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_test, prediction_test)
sns.heatmap(cm,
            annot=True,
            fmt="d",
            cmap='Blues',
            cbar=True,
            linewidths=0.5,
            linecolor='lightgray',
            xticklabels=['Predicted: No Churn', 'Predicted: Churn'],
            yticklabels=['True: No Churn', 'True: Churn'],
            annot_kws={"size": 12}
           )
plt.title("Random Forest CONFUSION MATRIX", fontsize=16, fontweight='bold')
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.tight_layout()
plt.show()

## Gradient Boosting Classifier

In [ ]:
gb = GradientBoostingClassifier(random_state=42)
gb.fit(X_train, y_train)
gb_pred = gb.predict(X_test)
print("Gradient Boosting Classifier", accuracy_score(y_test, gb_pred))

In [ ]:
print(classification_report(y_test, gb_pred))

In [ ]:
# Confusion Matrix
plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_test, gb_pred)
sns.heatmap(cm,
            annot=True,
            fmt="d",
            cmap='Blues',
            cbar=True,
            linewidths=0.5,
            linecolor='lightgray',
            xticklabels=['Predicted: No Churn', 'Predicted: Churn'],
            yticklabels=['True: No Churn', 'True: Churn'],
            annot_kws={"size": 12}
           )
plt.title("Gradient Boosting CONFUSION MATRIX", fontsize=16, fontweight='bold')
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.tight_layout()
plt.show()

# Ensembling Model

## Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier
clf1 = GradientBoostingClassifier()
clf2 = LogisticRegression()
clf3 = RandomForestClassifier()
eclf1 = VotingClassifier(estimators=[('gbc', clf1), ('lr', clf2), ('abc', clf3)], voting='soft')
eclf1.fit(X_train, y_train)
predictions = eclf1.predict(X_test)
print("Final Accuracy Score ")
print(accuracy_score(y_test, predictions))

# Hyperparameter Tuning

## GridSearchCV optimization for Logistic Regression

In [ ]:
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')
# parameter grid
parameters = {
    'penalty' : ['l1','l2'],
    'C'       : np.logspace(-3,3,7),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
}

In [ ]:
# We want to optimise our recall

model = LogisticRegression()
Grid = GridSearchCV(model,                    # model
                   param_grid = parameters,   # hyperparameters
                   scoring='f1',        # metric for scoring
                   cv=10)                     # number of folds

In [ ]:
Grid.fit(X_train_smote, y_train_smote)

In [ ]:
print("Best tuned Hyperparameters :", Grid.best_params_)
print("Accuracy :",Grid.best_score_)

## GridSearchCV optimization for Random Forest

In [ ]:
parameters_rf = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [4, 6, 8],
    'criterion': ['gini', 'entropy']
}

print("Random Forest hyperparameter grid defined:")
print(parameters_rf)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Instantiate RandomForestClassifier
model_rf_gs = RandomForestClassifier(random_state=42)

# Instantiate GridSearchCV
Grid_rf = GridSearchCV(model_rf_gs,                  # model
                       param_grid=parameters_rf,     # hyperparameters
                       scoring='f1',                 # metric for scoring
                       cv=5,                         # number of folds
                       n_jobs=-1,                    # Use all available cores
                       verbose=1)                     # print progress

# Fit GridSearchCV to the SMOTE-resampled training data
Grid_rf.fit(X_train_smote, y_train_smote)

print("Best tuned Hyperparameters for RandomForestClassifier:", Grid_rf.best_params_)
print("Best F1 Score for RandomForestClassifier:", Grid_rf.best_score_)

# Advanced Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# 1. Extract X and y from df_model (after feature selection and engineering)
X_final = df_model.drop(columns=['Churn'])
y_final = df_model['Churn']

# Identify numerical columns in X_final that need scaling
# Based on earlier feature engineering, these are likely the continuous ones
numerical_cols_for_scaling = ['tenure', 'Avg_Monthly_Spend', 'TotalCharges', 'MonthlyCharges']

def evaluate_advanced_validation(model, X, y, n_splits=5, random_state=42):
    """
    Performs Stratified K-Fold Cross-Validation with SMOTE on training folds,
    scaling, and reports Mean/STD Recall.

    Args:
        model: The machine learning model to evaluate.
        X (pd.DataFrame): Feature DataFrame.
        y (pd.Series): Target Series.
        n_splits (int): Number of folds for cross-validation.
        random_state (int): Random state for reproducibility.

    Returns:
        tuple: (mean_recall, std_recall)
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    recall_scores = []

    print(f"Evaluating model: {model.__class__.__name__} with {n_splits}-Fold CV...")

    # Identify numerical columns within the current X to apply scaling
    # Ensure this list aligns with actual numerical columns in X
    current_numerical_cols = [col for col in numerical_cols_for_scaling if col in X.columns]

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

        # Apply StandardScaler to numerical columns within the fold
        scaler = StandardScaler()
        if current_numerical_cols:
            X_train_fold[current_numerical_cols] = scaler.fit_transform(X_train_fold[current_numerical_cols])
            X_val_fold[current_numerical_cols] = scaler.transform(X_val_fold[current_numerical_cols])

        # Apply SMOTE to the training data
        smote = SMOTE(random_state=random_state)
        X_train_smote_fold, y_train_smote_fold = smote.fit_resample(X_train_fold, y_train_fold)

        # Train the model
        model.fit(X_train_smote_fold, y_train_smote_fold)

        # Make predictions on the validation set
        y_pred_fold = model.predict(X_val_fold)

        # Calculate Recall for the positive class (churn)
        recall = recall_score(y_val_fold, y_pred_fold)
        recall_scores.append(recall)
        print(f"  Fold {fold+1}/{n_splits} Recall: {recall:.4f}")

    mean_recall = np.mean(recall_scores)
    std_recall = np.std(recall_scores)

    print(f"\n{model.__class__.__name__} - Mean Recall: {mean_recall:.4f} (Std: {std_recall:.4f})")
    return mean_recall, std_recall

# 2. Get the best models from GridSearchCV and other existing models
best_lr_model = Grid.best_estimator_
best_rf_model = Grid_rf.best_estimator_
# Gradient Boosting Classifier is already initialized as 'gb'
# Voting Classifier is already initialized as 'eclf1'

# 3. Evaluate the best Logistic Regression model
print("\n" + "="*70)
print("Advanced Validation for Tuned Logistic Regression")
print("="*70)
lr_mean_recall, lr_std_recall = evaluate_advanced_validation(best_lr_model, X_final, y_final)

# 4. Evaluate the best Random Forest model
print("\n" + "="*70)
print("Advanced Validation for Tuned Random Forest Classifier")
print("="*70)
rf_mean_recall, rf_std_recall = evaluate_advanced_validation(best_rf_model, X_final, y_final)

# 5. Evaluate the Gradient Boosting Classifier
print("\n" + "="*70)
print("Advanced Validation for Gradient Boosting Classifier")
print("="*70)
gb_mean_recall, gb_std_recall = evaluate_advanced_validation(gb, X_final, y_final)

# 6. Evaluate the Voting Classifier
print("\n" + "="*70)
print("Advanced Validation for Voting Classifier")
print("="*70)
voting_mean_recall, voting_std_recall = evaluate_advanced_validation(eclf1, X_final, y_final)

# Optional: Compare results
print("\n" + "="*70)
print("Advanced Validation Summary (Mean Recall)")
print("="*70)
print(f"Logistic Regression  : {lr_mean_recall:.4f} (Std: {lr_std_recall:.4f})")
print(f"Random Forest        : {rf_mean_recall:.4f} (Std: {rf_std_recall:.4f})")
print(f"Gradient Boosting    : {gb_mean_recall:.4f} (Std: {gb_std_recall:.4f})")
print(f"Voting Classifier    : {voting_mean_recall:.4f} (Std: {voting_std_recall:.4f})")


## ROC

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))

# Logistic Regression
y_pred_prob_lr = lr_model.predict_proba(X_test)[:, 1]
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_pred_prob_lr)
auc_lr = roc_auc_score(y_test, y_pred_prob_lr)
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC = {auc_lr:.2f})', linewidth=2)

# KNN
y_pred_prob_knn = knn_model.predict_proba(X_test)[:, 1]
fpr_knn, tpr_knn, _ = roc_curve(y_test, y_pred_prob_knn)
auc_knn = roc_auc_score(y_test, y_pred_prob_knn)
plt.plot(fpr_knn, tpr_knn, label=f'KNN (AUC = {auc_knn:.2f})', linewidth=2)

# Random Forest
y_pred_prob_rf = model_rf.predict_proba(X_test)[:, 1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_prob_rf)
auc_rf = roc_auc_score(y_test, y_pred_prob_rf)
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {auc_rf:.2f})', linewidth=2)

# Gradient Boosting Classifier
y_pred_prob_gb = gb.predict_proba(X_test)[:, 1]
fpr_gb, tpr_gb, _ = roc_curve(y_test, y_pred_prob_gb)
auc_gb = roc_auc_score(y_test, y_pred_prob_gb)
plt.plot(fpr_gb, tpr_gb, label=f'Gradient Boosting (AUC = {auc_gb:.2f})', linewidth=2)

# Plot the random chance line
plt.plot([0, 1], [0, 1], 'k--', label='Random Chance', linestyle='--')

plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves Comparison', fontsize=16, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## Confusion Matrices

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

# Define the models and their predictions
models = {
    "Logistic Regression": lr_model.predict(X_test),
    "KNN": knn_model.predict(X_test),
    "Random Forest": model_rf.predict(X_test),
    "Gradient Boosting": gb.predict(X_test)
}

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for i, (name, y_pred) in enumerate(models.items()):
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Churn', 'Churn'])
    disp.plot(cmap='Blues', ax=axes[i], colorbar=False)
    axes[i].set_title(f'{name} Confusion Matrix', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Predicted Label', fontsize=12)
    axes[i].set_ylabel('True Label', fontsize=12)

plt.suptitle('Comparison of Confusion Matrices', fontsize=18, fontweight='bold', y=1.02)
plt.tight_layout(rect=[0, 0, 1, 0.98]) # Adjust layout to prevent suptitle overlap
plt.show()

## Tune Logistic Regression for Recall



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

# 1. Initialize a LogisticRegression model
model_lr_recall = LogisticRegression(random_state=42, max_iter=1000)

# 2. Create a GridSearchCV object
# Using the 'parameters' dictionary defined earlier
Grid_lr_recall = GridSearchCV(model_lr_recall,                    # model
                              param_grid = parameters,           # hyperparameters
                              scoring='recall',                  # metric for scoring - optimize for recall
                              cv=10)                             # number of folds

# 3. Fit Grid_lr_recall to the SMOTE-resampled training data
Grid_lr_recall.fit(X_train_smote, y_train_smote)

# 4. Print the best hyperparameters found
print("Best tuned Hyperparameters for Logistic Regression (Recall):", Grid_lr_recall.best_params_)

# 5. Print the best recall score achieved
print("Best Recall Score for Logistic Regression:", Grid_lr_recall.best_score_)


## Evaluate Tuned Logistic Regression Model



In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, recall_score, f1_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Re-establish X, y from df_model (which contains the final selected features)
# Assuming df_model and final_features are available from earlier successful cells.
X = df_model.drop(columns=['Churn'])
y = df_model['Churn']

# Re-perform train-test split on the feature-selected data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=40, stratify=y
)

# Identify numerical columns within the selected features for scaling
numerical_cols_for_scaling_selected = ['tenure', 'Avg_Monthly_Spend', 'TotalCharges', 'MonthlyCharges']
current_numerical_cols = [col for col in numerical_cols_for_scaling_selected if col in X_train.columns]

# Apply StandardScaler to the numerical columns of the newly split X_train and X_test
scaler = StandardScaler()
if current_numerical_cols:
    X_train[current_numerical_cols] = scaler.fit_transform(X_train[current_numerical_cols])
    X_test[current_numerical_cols] = scaler.transform(X_test[current_numerical_cols])

# Re-generate X_train_smote and y_train_smote from the now-correct X_train and y_train
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Re-define 'parameters' for GridSearchCV (as in TqRC3yUqDQ7p)
parameters = {
    'penalty' : ['l1','l2'],
    'C'       : np.logspace(-3,3,7),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
}

# Re-initialize and re-fit Grid_lr_recall (as in 249b0c2b) to ensure consistency
model_lr_recall = LogisticRegression(random_state=42, max_iter=1000)
Grid_lr_recall = GridSearchCV(model_lr_recall,                    # model
                              param_grid = parameters,           # hyperparameters
                              scoring='recall',                  # metric for scoring - optimize for recall
                              cv=10)                             # number of folds
Grid_lr_recall.fit(X_train_smote, y_train_smote)

# Now, proceed with the original content of this cell to evaluate
# Get the best Logistic Regression model from GridSearchCV
best_lr_model_recall = Grid_lr_recall.best_estimator_

# Make predictions on the test set
y_pred_lr_recall = best_lr_model_recall.predict(X_test)

# Print the classification report
print("\nClassification Report for Tuned Logistic Regression (Optimized for Recall):\n")
report_lr_recall = classification_report(y_test, y_pred_lr_recall, output_dict=True)
print(classification_report(y_test, y_pred_lr_recall))

# Extract and print recall and F1-score for the churn class (class 1)
recall_churn_lr = report_lr_recall['1']['recall']
f1_churn_lr = report_lr_recall['1']['f1-score']

print(f"\nRecall for Churn (class 1): {recall_churn_lr:.4f}")
print(f"F1-Score for Churn (class 1): {f1_churn_lr:.4f}")

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, recall_score, f1_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Re-establish X, y from df_model (which contains the final selected features)
# Assuming df_model and final_features are available from earlier successful cells.
X = df_model.drop(columns=['Churn'])
y = df_model['Churn']

# Re-perform train-test split on the feature-selected data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=40, stratify=y
)

# Identify numerical columns within the selected features for scaling
numerical_cols_for_scaling_selected = ['tenure', 'Avg_Monthly_Spend', 'TotalCharges', 'MonthlyCharges']
current_numerical_cols = [col for col in numerical_cols_for_scaling_selected if col in X_train.columns]

# Apply StandardScaler to the numerical columns of the newly split X_train and X_test
scaler = StandardScaler()
if current_numerical_cols:
    X_train[current_numerical_cols] = scaler.fit_transform(X_train[current_numerical_cols])
    X_test[current_numerical_cols] = scaler.transform(X_test[current_numerical_cols])

# Re-generate X_train_smote and y_train_smote from the now-correct X_train and y_train
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Re-define 'parameters' for GridSearchCV (as in TqRC3yUqDQ7p)
parameters = {
    'penalty' : ['l1','l2'],
    'C'       : np.logspace(-3,3,7),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
}

# Re-initialize and re-fit Grid_lr_recall (as in 249b0c2b) to ensure consistency
model_lr_recall = LogisticRegression(random_state=42, max_iter=1000)
Grid_lr_recall = GridSearchCV(model_lr_recall,                    # model
                              param_grid = parameters,           # hyperparameters
                              scoring='recall',                  # metric for scoring - optimize for recall
                              cv=10)                             # number of folds
Grid_lr_recall.fit(X_train_smote, y_train_smote)

# Now, proceed with the original content of this cell to evaluate
# Get the best Logistic Regression model from GridSearchCV
best_lr_model_recall = Grid_lr_recall.best_estimator_

# Make predictions on the test set
y_pred_lr_recall = best_lr_model_recall.predict(X_test)

# Print the classification report
print("\nClassification Report for Tuned Logistic Regression (Optimized for Recall):\n")
report_lr_recall = classification_report(y_test, y_pred_lr_recall, output_dict=True)
print(classification_report(y_test, y_pred_lr_recall))

# Extract and print recall and F1-score for the churn class (class 1)
recall_churn_lr = report_lr_recall['1']['recall']
f1_churn_lr = report_lr_recall['1']['f1-score']

print(f"\nRecall for Churn (class 1): {recall_churn_lr:.4f}")
print(f"F1-Score for Churn (class 1): {f1_churn_lr:.4f}")


## Tune Random Forest for Recall




In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# 1. Initialize a RandomForestClassifier
model_rf_recall = RandomForestClassifier(random_state=42)

# 2. Create a GridSearchCV object
# Using the 'parameters_rf' dictionary defined earlier (from cell 804b1f7b)
Grid_rf_recall = GridSearchCV(model_rf_recall,                  # model
                              param_grid=parameters_rf,     # hyperparameters
                              scoring='recall',                 # metric for scoring - optimize for recall
                              cv=5,                         # number of folds
                              n_jobs=-1,                    # Use all available cores
                              verbose=1)                     # print progress

# 3. Fit GridSearchCV to the SMOTE-resampled training data
Grid_rf_recall.fit(X_train_smote, y_train_smote)

# 4. Print the best hyperparameters found
print("Best tuned Hyperparameters for RandomForestClassifier (Recall):", Grid_rf_recall.best_params_)

# 5. Print the best recall score achieved
print("Best Recall Score for RandomForestClassifier:", Grid_rf_recall.best_score_)

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, recall_score, f1_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Re-establish X, y from df_model (which contains the final selected features)
# Assuming df_model and final_features are available from earlier successful cells.
X = df_model.drop(columns=['Churn'])
y = df_model['Churn']

# Re-perform train-test split on the feature-selected data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=40, stratify=y
)

# Identify numerical columns within the selected features for scaling
numerical_cols_for_scaling_selected = ['tenure', 'Avg_Monthly_Spend', 'TotalCharges', 'MonthlyCharges']
current_numerical_cols = [col for col in numerical_cols_for_scaling_selected if col in X_train.columns]

# Apply StandardScaler to the numerical columns of the newly split X_train and X_test
scaler = StandardScaler()
if current_numerical_cols:
    X_train[current_numerical_cols] = scaler.fit_transform(X_train[current_numerical_cols])
    X_test[current_numerical_cols] = scaler.transform(X_test[current_numerical_cols])

# Re-generate X_train_smote and y_train_smote from the now-correct X_train and y_train
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Re-define 'parameters_rf' for GridSearchCV (as in 804b1f7b)
parameters_rf = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [4, 6, 8],
    'criterion': ['gini', 'entropy']
}

# Re-initialize and re-fit Grid_rf_recall (as in 0e79c9c8) to ensure consistency
model_rf_recall = RandomForestClassifier(random_state=42)
Grid_rf_recall = GridSearchCV(model_rf_recall,                  # model
                              param_grid=parameters_rf,     # hyperparameters
                              scoring='recall',                 # metric for scoring - optimize for recall
                              cv=5,                         # number of folds
                              n_jobs=-1,                    # Use all available cores
                              verbose=0)                     # set verbose to 0 to avoid repeated output during re-fit
Grid_rf_recall.fit(X_train_smote, y_train_smote)

# Get the best Random Forest model from GridSearchCV
best_rf_model_recall = Grid_rf_recall.best_estimator_

# Make predictions on the test set
y_pred_rf_recall = best_rf_model_recall.predict(X_test)

# Print the classification report
print("\nClassification Report for Tuned Random Forest (Optimized for Recall):\n")
report_rf_recall = classification_report(y_test, y_pred_rf_recall, output_dict=True)
print(classification_report(y_test, y_pred_rf_recall))

# Extract and print recall and F1-score for the churn class (class 1)
recall_churn_rf = report_rf_recall['1']['recall']
f1_churn_rf = report_rf_recall['1']['f1-score']

print(f"\nRecall for Churn (class 1): {recall_churn_rf:.4f}")
print(f"F1-Score for Churn (class 1): {f1_churn_rf:.4f}")

### Summary of Tuned Random Forest Results

The `RandomForestClassifier` was tuned using `GridSearchCV` to optimize for **recall** on the SMOTE-resampled training data. The best hyperparameters found were:

*   `max_depth`: 8
*   `max_features`: 'sqrt'
*   `n_estimators`: 100
*   `criterion`: 'gini'

On the unseen test set, this optimized model achieved a **recall for Churn (class 1)** of **0.8449** and an **F1-Score for Churn (class 1)** of **0.6558**. This indicates a strong ability to identify customers who churn, which is crucial in churn prediction scenarios where minimizing false negatives (missing actual churners) is important.

## Compare and Select Best Model

### Subtask:
Compare the recall-optimized Logistic Regression and Random Forest models and select the best one based on the highest recall, while ensuring an F1-score of at least 50% for the churn class (class 1).


**Reasoning**:
I need to compare the performance metrics (recall and F1-score for the churn class) of the two tuned models (Logistic Regression and Random Forest) as per the instructions, apply the given conditions, and then identify the best performing model.



In [ ]:
print('--- Comparison of Recall-Optimized Models ---\n')
print(f"Logistic Regression (Churn Class): Recall = {recall_churn_lr:.4f}, F1-Score = {f1_churn_lr:.4f}")
print(f"Random Forest (Churn Class):       Recall = {recall_churn_rf:.4f}, F1-Score = {f1_churn_rf:.4f}\n")

best_model_name = "None"
highest_recall = -1

# Check Logistic Regression
if f1_churn_lr >= 0.50:
    print("Logistic Regression meets F1-score criteria.")
    if recall_churn_lr > highest_recall:
        highest_recall = recall_churn_lr
        best_model_name = "Logistic Regression"

# Check Random Forest
if f1_churn_rf >= 0.50:
    print("Random Forest meets F1-score criteria.")
    if recall_churn_rf > highest_recall:
        highest_recall = recall_churn_rf
        best_model_name = "Random Forest"
    elif recall_churn_rf == highest_recall and best_model_name == "None": # If both have same recall and LR didn't meet F1, RF is the first to meet criteria
        best_model_name = "Random Forest" # Prioritize RF if LR didn't meet F1-score criteria


if best_model_name != "None":
    print(f"\nBased on the criteria, the best model is: {best_model_name} with Recall = {highest_recall:.4f}")
    selected_best_model = best_lr_model_recall if best_model_name == "Logistic Regression" else best_rf_model_recall
else:
    print("\nNo model met the F1-score threshold of 0.50 for the churn class.")
    selected_best_model = None


# Business Suggestion (Using EV framework)


Calculate churn probabilities using the `selected_best_model`, then implement an Expected Value (EV) framework with illustrative business parameters (e.g., Cost of Churn, Cost of Intervention, Probability of Success of intervention) to evaluate and recommend optimal discount strategies for customer retention, including visualization of the financial impact.

## Calculate Churn Probabilities


Use the `selected_best_model` (which is `best_rf_model_recall` in this case) to calculate churn probabilities for the test set.


In [ ]:
churn_probabilities = selected_best_model.predict_proba(X_test)[:, 1]
print("Churn Probabilities calculated successfully. First 5 probabilities:")
print(churn_probabilities[:5])


Now that churn probabilities are calculated, the next step is to define the business parameters for the Expected Value (EV) framework. This includes costs associated with churn, intervention costs, and the success rate of interventions. These parameters are crucial for evaluating the financial impact of different retention strategies.



In [ ]:
import pandas as pd
import numpy as np

# --- Define illustrative business parameters (These can be adjusted) ---
COST_OF_CHURN = 500  # Average estimated loss of revenue/profit per churned customer
COST_OF_INTERVENTION_HIGH = 100 # Cost of a high-value intervention (e.g., 20% discount)
COST_OF_INTERVENTION_LOW = 50   # Cost of a low-value intervention (e.g., 10% discount)

# Probability of success of intervention given the customer was predicted to churn
# (These are estimates and would ideally come from A/B testing or historical data)
PROB_SUCCESS_HIGH_INTERVENTION = 0.60 # Higher success rate for high discount
PROB_SUCCESS_LOW_INTERVENTION = 0.30  # Lower success rate for low discount

print("Business parameters defined for EV framework.")


With churn probabilities and business parameters defined, the next step is to calculate the Expected Value (EV) for different intervention strategies for each customer. This will allow for a direct comparison of the financial impact of each strategy.



In [ ]:
customer_data = X_test.copy()
customer_data['churn_prob'] = churn_probabilities
customer_data['true_churn'] = y_test.values

# --- Expected Value Calculation for each strategy ---

# 1. EV for 'No Intervention' strategy
customer_data['EV_No_Intervention'] = -customer_data['churn_prob'] * COST_OF_CHURN

# 2. EV for 'Low-Cost Intervention' strategy
# Cost of intervention if we intervene (always if prob > threshold)
# Benefit if intervention is successful: (1 - churn_prob) * COST_OF_CHURN
# Cost if intervention fails: COST_OF_CHURN
# Net benefit/cost: (PROB_SUCCESS * COST_OF_CHURN) - COST_OF_INTERVENTION_LOW
customer_data['EV_Low_Intervention'] = (customer_data['churn_prob'] * (PROB_SUCCESS_LOW_INTERVENTION * COST_OF_CHURN - COST_OF_INTERVENTION_LOW)) + \
                                       ((1 - customer_data['churn_prob']) * (-COST_OF_INTERVENTION_LOW))

# 3. EV for 'High-Cost Intervention' strategy
customer_data['EV_High_Intervention'] = (customer_data['churn_prob'] * (PROB_SUCCESS_HIGH_INTERVENTION * COST_OF_CHURN - COST_OF_INTERVENTION_HIGH)) + \
                                        ((1 - customer_data['churn_prob']) * (-COST_OF_INTERVENTION_HIGH))

print("Expected Value for different intervention strategies calculated for each customer.")
print("First 5 rows of customer_data with EV calculations:")
print(customer_data[['churn_prob', 'true_churn', 'EV_No_Intervention', 'EV_Low_Intervention', 'EV_High_Intervention']].head())


With the Expected Values for each strategy calculated per customer, the next step is to determine the optimal strategy for each customer by selecting the one with the highest EV, and then summarize the total expected value across all customers for each strategy to assess overall financial impact.



In [ ]:
intervention_cols = ['EV_No_Intervention', 'EV_Low_Intervention', 'EV_High_Intervention']

customer_data['Optimal_Strategy'] = customer_data[intervention_cols].idxmax(axis=1)
customer_data['Optimal_EV'] = customer_data[intervention_cols].max(axis=1)

# --- Summarize the total expected value for each strategy ---

total_ev_no_intervention = customer_data['EV_No_Intervention'].sum()
total_ev_low_intervention = customer_data['EV_Low_Intervention'].sum()
total_ev_high_intervention = customer_data['EV_High_Intervention'].sum()

total_optimal_ev = customer_data['Optimal_EV'].sum()

print("Total Expected Value for 'No Intervention': ", f"${total_ev_no_intervention:,.2f}")
print("Total Expected Value for 'Low-Cost Intervention': ", f"${total_ev_low_intervention:,.2f}")
print("Total Expected Value for 'High-Cost Intervention': ", f"${total_ev_high_intervention:,.2f}")
print("Total Expected Value with Optimal Strategy: ", f"${total_optimal_ev:,.2f}")

print("\nFirst 5 rows of customer_data with optimal strategy selection:")
print(customer_data[['churn_prob', 'EV_No_Intervention', 'EV_Low_Intervention', 'EV_High_Intervention', 'Optimal_Strategy', 'Optimal_EV']].head())


To better understand the recommendations from the Expected Value framework, it's essential to visualize the distribution of optimal strategies among customers and compare the total financial impact of each strategy. This will provide a clear business recommendation.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Distribution of Optimal Strategies ---
plt.figure(figsize=(8, 6))
sns.countplot(x='Optimal_Strategy', data=customer_data, palette='viridis')
plt.title('Distribution of Optimal Intervention Strategies', fontsize=14, fontweight='bold')
plt.xlabel('Optimal Strategy', fontsize=12)
plt.ylabel('Number of Customers', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# --- 2. Total Expected Value Comparison ---

total_ev_summary = pd.DataFrame({
    'Strategy': ['No Intervention', 'Low-Cost Intervention', 'High-Cost Intervention', 'Optimal Combined Strategy'],
    'Total EV': [total_ev_no_intervention, total_ev_low_intervention, total_ev_high_intervention, total_optimal_ev]
})

plt.figure(figsize=(10, 7))
sns.barplot(x='Strategy', y='Total EV', data=total_ev_summary, palette='coolwarm')
plt.title('Total Expected Value for Different Retention Strategies', fontsize=14, fontweight='bold')
plt.xlabel('Strategy', fontsize=12)
plt.ylabel('Total Expected Value ($)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# --- 3. Histogram of Churn Probabilities ---
plt.figure(figsize=(8, 6))
sns.histplot(customer_data['churn_prob'], bins=30, kde=True, color='skyblue')
plt.title('Distribution of Predicted Churn Probabilities', fontsize=14, fontweight='bold')
plt.xlabel('Churn Probability', fontsize=12)
plt.ylabel('Number of Customers', fontsize=12)
plt.tight_layout()
plt.show()


## Present Business Recommendations

### Subtask:
Present the results of the Expected Value analysis, highlighting the most profitable discount strategy and providing actionable business recommendations based on the findings.


## Business Recommendations from Expected Value Analysis

Based on the Expected Value (EV) framework and the analysis of various intervention strategies, here are the key findings and actionable business recommendations:

### 1. Analysis of Total Expected Value

Let's revisit the total expected values for each strategy:

*   **No Intervention**: `${-452433.74:,.2f}`
    *   This represents the total potential loss if no retention efforts are made for any customer. It's a significant negative value, highlighting the high cost of churn.

*   **Low-Cost Intervention**: `${30230.12:,.2f}`
    *   Implementing a low-cost intervention (e.g., a 10% discount) for all customers with a churn probability above a certain threshold (if applied universally) results in a positive expected value, indicating a net financial gain compared to no intervention.

*   **High-Cost Intervention**: `${60460.24:,.2f}`
    *   Implementing a high-cost intervention (e.g., a 20% discount) for all customers with a churn probability above a certain threshold (if applied universally) yields an even higher positive expected value, suggesting that for certain customers, a more significant discount is more profitable.

*   **Optimal Combined Strategy**: `${87502.83:,.2f}`
    *   This is the most profitable strategy. By tailoring the intervention (No Intervention, Low-Cost, or High-Cost) to each customer based on their individual churn probability and the expected value of each action, the total expected value is maximized.

### 2. Distribution of Optimal Strategies

The `Distribution of Optimal Intervention Strategies` plot shows how many customers fall into each recommended category:

*   A large portion of customers might still be recommended for 'No Intervention', either because their churn probability is low, or the cost of intervention outweighs the potential benefit.
*   A significant number of customers are assigned to 'EV_Low_Intervention' and 'EV_High_Intervention', indicating that targeted discounts are financially beneficial for these segments.

### 3. Business Recommendation: Implement an Adaptive, Optimal Combined Strategy

**Recommendation**: The company should implement a dynamic, customer-specific retention strategy based on the 'Optimal Combined Strategy' identified by the Expected Value framework. This involves:

1.  **Predicting Churn Probability**: For each customer, use the `selected_best_model` (Random Forest, in this case) to predict their churn probability.
2.  **Calculating Expected Value for Interventions**: For each customer, calculate the Expected Value of 'No Intervention', 'Low-Cost Intervention', and 'High-Cost Intervention' using the defined business parameters.
3.  **Selecting the Optimal Action**: For each customer, choose the intervention strategy that yields the highest Expected Value.

### 4. Financial Implications

Implementing the **Optimal Combined Strategy** is projected to generate an additional expected value of approximately **`$87,502.83`**. This is a substantial improvement compared to the `$-452,433.74` expected loss if **No Intervention** is undertaken, representing a potential swing of over **`$500,000`**.

This framework ensures that resources (discounts) are allocated efficiently, targeting customers where the intervention is most likely to yield a positive financial return, and avoiding costly interventions for customers unlikely to churn or unlikely to be retained.

### 5. Optimizing Customer Retention Efforts

*   **Targeted Marketing**: Instead of generic marketing campaigns, retention efforts can be highly targeted. Customers identified for 'High-Cost Intervention' can receive personalized offers, while those for 'Low-Cost Intervention' can get more standard discounts.
*   **Resource Allocation**: The framework guides where to invest marketing and customer service resources, focusing on customers with high churn probability and a positive EV for intervention.
*   **Continuous Improvement**: The business parameters (Cost of Churn, Cost of Intervention, Probability of Success) should be regularly reviewed and updated based on new data, A/B test results, and market conditions. This allows the model to adapt and improve its recommendations over time.
*   **Feedback Loop**: Track the actual outcomes of interventions. Did customers who received a discount churn less? Was the predicted success rate accurate? Use this data to refine the `PROB_SUCCESS` parameters.

By adopting this data-driven approach, the company can move from reactive customer retention to a proactive, financially optimized strategy, significantly impacting the bottom line.

## Business Recommendations from Expected Value Analysis

Based on the Expected Value (EV) framework and the analysis of various intervention strategies, here are the key findings and actionable business recommendations:

### 1. Analysis of Total Expected Value

Let's revisit the total expected values for each strategy:

*   **No Intervention**: `$-452,433.74`
    *   This represents the total potential loss if no retention efforts are made for any customer. It's a significant negative value, highlighting the high cost of churn.

*   **Low-Cost Intervention**: `$-30,230.12`
    *   Implementing a low-cost intervention (e.g., a 10% discount) for all customers with a churn probability above a certain threshold (if applied universally) results in a positive expected value, indicating a net financial gain compared to no intervention.

*   **High-Cost Intervention**: `$-60,460.24`
    *   Implementing a high-cost intervention (e.g., a 20% discount) for all customers with a churn probability above a certain threshold (if applied universally) yields an even higher positive expected value, suggesting that for certain customers, a more significant discount is more profitable.

*   **Optimal Combined Strategy**: `$-87,502.83`
    *   This is the most profitable strategy. By tailoring the intervention (No Intervention, Low-Cost, or High-Cost) to each customer based on their individual churn probability and the expected value of each action, the total expected value is maximized.

### 2. Distribution of Optimal Strategies

The `Distribution of Optimal Intervention Strategies` plot shows how many customers fall into each recommended category:

*   A large portion of customers might still be recommended for 'No Intervention', either because their churn probability is low, or the cost of intervention outweighs the potential benefit.
*   A significant number of customers are assigned to 'EV_Low_Intervention' and 'EV_High_Intervention', indicating that targeted discounts are financially beneficial for these segments.

### 3. Business Recommendation: Implement an Adaptive, Optimal Combined Strategy

**Recommendation**: The company should implement a dynamic, customer-specific retention strategy based on the 'Optimal Combined Strategy' identified by the Expected Value framework. This involves:

1.  **Predicting Churn Probability**: For each customer, use the `selected_best_model` (Random Forest, in this case) to predict their churn probability.
2.  **Calculating Expected Value for Interventions**: For each customer, calculate the Expected Value of 'No Intervention', 'Low-Cost Intervention', and 'High-Cost Intervention' using the defined business parameters.
3.  **Selecting the Optimal Action**: For each customer, choose the intervention strategy that yields the highest Expected Value.

### 4. Financial Implications

Implementing the **Optimal Combined Strategy** is projected to generate an additional expected value of approximately **`$87,502.83`**. This is a substantial improvement compared to the `$-452,433.74` expected loss if **No Intervention** is undertaken, representing a potential swing of over **`$500,000`**.

This framework ensures that resources (discounts) are allocated efficiently, targeting customers where the intervention is most likely to yield a positive financial return, and avoiding costly interventions for customers unlikely to churn or unlikely to be retained.

### 5. Optimizing Customer Retention Efforts

*   **Targeted Marketing**: Instead of generic marketing campaigns, retention efforts can be highly targeted. Customers identified for 'High-Cost Intervention' can receive personalized offers, while those for 'Low-Cost Intervention' can get more standard discounts.
*   **Resource Allocation**: The framework guides where to invest marketing and customer service resources, focusing on customers with high churn probability and a positive EV for intervention.
*   **Continuous Improvement**: The business parameters (Cost of Churn, Cost of Intervention, Probability of Success) should be regularly reviewed and updated based on new data, A/B test results, and market conditions. This allows the model to adapt and improve its recommendations over time.
*   **Feedback Loop**: Track the actual outcomes of interventions. Did customers who received a discount churn less? Was the predicted success rate accurate? Use this data to refine the `PROB_SUCCESS` parameters.

By adopting this data-driven approach, the company can move from reactive customer retention to a proactive, financially optimized strategy, significantly impacting the bottom line.


## Business Recommendations from Expected Value Analysis

Based on the Expected Value (EV) framework and the analysis of various intervention strategies, here are the key findings and actionable business recommendations:

### 1. Analysis of Total Expected Value

Let's revisit the total expected values for each strategy:

*   **No Intervention**: `$-452,433.74`
    *   This represents the total potential loss if no retention efforts are made for any customer. It's a significant negative value, highlighting the high cost of churn.

*   **Low-Cost Intervention**: `$-30,230.12`
    *   Implementing a low-cost intervention (e.g., a 10% discount) for all customers with a churn probability above a certain threshold (if applied universally) results in a positive expected value, indicating a net financial gain compared to no intervention.

*   **High-Cost Intervention**: `$-60,460.24`
    *   Implementing a high-cost intervention (e.g., a 20% discount) for all customers with a churn probability above a certain threshold (if applied universally) yields an even higher positive expected value, suggesting that for certain customers, a more significant discount is more profitable.

*   **Optimal Combined Strategy**: `$-87,502.83`
    *   This is the most profitable strategy. By tailoring the intervention (No Intervention, Low-Cost, or High-Cost) to each customer based on their individual churn probability and the expected value of each action, the total expected value is maximized.

### 2. Distribution of Optimal Strategies

The `Distribution of Optimal Intervention Strategies` plot shows how many customers fall into each recommended category:

*   A large portion of customers might still be recommended for 'No Intervention', either because their churn probability is low, or the cost of intervention outweighs the potential benefit.
*   A significant number of customers are assigned to 'EV_Low_Intervention' and 'EV_High_Intervention', indicating that targeted discounts are financially beneficial for these segments.

### 3. Business Recommendation: Implement an Adaptive, Optimal Combined Strategy

**Recommendation**: The company should implement a dynamic, customer-specific retention strategy based on the 'Optimal Combined Strategy' identified by the Expected Value framework. This involves:

1.  **Predicting Churn Probability**: For each customer, use the `selected_best_model` (Random Forest, in this case) to predict their churn probability.
2.  **Calculating Expected Value for Interventions**: For each customer, calculate the Expected Value of 'No Intervention', 'Low-Cost Intervention', and 'High-Cost Intervention' using the defined business parameters.
3.  **Selecting the Optimal Action**: For each customer, choose the intervention strategy that yields the highest Expected Value.

### 4. Financial Implications

Implementing the **Optimal Combined Strategy** is projected to generate an additional expected value of approximately **`$87,502.83`**. This is a substantial improvement compared to the `$-452,433.74` expected loss if **No Intervention** is undertaken, representing a potential swing of over **`$500,000`**.

This framework ensures that resources (discounts) are allocated efficiently, targeting customers where the intervention is most likely to yield a positive financial return, and avoiding costly interventions for customers unlikely to churn or unlikely to be retained.

### 5. Optimizing Customer Retention Efforts

*   **Targeted Marketing**: Instead of generic marketing campaigns, retention efforts can be highly targeted. Customers identified for 'High-Cost Intervention' can receive personalized offers, while those for 'Low-Cost Intervention' can get more standard discounts.
*   **Resource Allocation**: The framework guides where to invest marketing and customer service resources, focusing on customers with high churn probability and a positive EV for intervention.
*   **Continuous Improvement**: The business parameters (Cost of Churn, Cost of Intervention, Probability of Success) should be regularly reviewed and updated based on new data, A/B test results, and market conditions. This allows the model to adapt and improve its recommendations over time.
*   **Feedback Loop**: Track the actual outcomes of interventions. Did customers who received a discount churn less? Was the predicted success rate accurate? Use this data to refine the `PROB_SUCCESS` parameters.

By adopting this data-driven approach, the company can move from reactive customer retention to a proactive, financially optimized strategy, significantly impacting the bottom line.


## Business Recommendations from Expected Value Analysis

Based on the Expected Value (EV) framework and the analysis of various intervention strategies, here are the key findings and actionable business recommendations:

### 1. Analysis of Total Expected Value

Let's revisit the total expected values for each strategy:

*   **No Intervention**: `$-452,433.74`
    *   This represents the total potential loss if no retention efforts are made for any customer. It's a significant negative value, highlighting the high cost of churn.

*   **Low-Cost Intervention**: `$30,230.12`
    *   Implementing a low-cost intervention (e.g., a 10% discount) for all customers with a churn probability above a certain threshold (if applied universally) results in a positive expected value, indicating a net financial gain compared to no intervention.

*   **High-Cost Intervention**: `$60,460.24`
    *   Implementing a high-cost intervention (e.g., a 20% discount) for all customers with a churn probability above a certain threshold (if applied universally) yields an even higher positive expected value, suggesting that for certain customers, a more significant discount is more profitable.

*   **Optimal Combined Strategy**: `$87,502.83`
    *   This is the most profitable strategy. By tailoring the intervention (No Intervention, Low-Cost, or High-Cost) to each customer based on their individual churn probability and the expected value of each action, the total expected value is maximized.

### 2. Distribution of Optimal Strategies

The `Distribution of Optimal Intervention Strategies` plot shows how many customers fall into each recommended category:

*   A large portion of customers might still be recommended for 'No Intervention', either because their churn probability is low, or the cost of intervention outweighs the potential benefit.
*   A significant number of customers are assigned to 'EV_Low_Intervention' and 'EV_High_Intervention', indicating that targeted discounts are financially beneficial for these segments.

### 3. Business Recommendation: Implement an Adaptive, Optimal Combined Strategy

**Recommendation**: The company should implement a dynamic, customer-specific retention strategy based on the 'Optimal Combined Strategy' identified by the Expected Value framework. This involves:

1.  **Predicting Churn Probability**: For each customer, use the `selected_best_model` (Random Forest, in this case) to predict their churn probability.
2.  **Calculating Expected Value for Interventions**: For each customer, calculate the Expected Value of 'No Intervention', 'Low-Cost Intervention', and 'High-Cost Intervention' using the defined business parameters.
3.  **Selecting the Optimal Action**: For each customer, choose the intervention strategy that yields the highest Expected Value.

### 4. Financial Implications

Implementing the **Optimal Combined Strategy** is projected to generate an additional expected value of approximately **`$87,502.83`**. This is a substantial improvement compared to the `$-452,433.74` expected loss if **No Intervention** is undertaken, representing a potential swing of over **`$500,000`**.

This framework ensures that resources (discounts) are allocated efficiently, targeting customers where the intervention is most likely to yield a positive financial return, and avoiding costly interventions for customers unlikely to churn or unlikely to be retained.

### 5. Optimizing Customer Retention Efforts

*   **Targeted Marketing**: Instead of generic marketing campaigns, retention efforts can be highly targeted. Customers identified for 'High-Cost Intervention' can receive personalized offers, while those for 'Low-Cost Intervention' can get more standard discounts.
*   **Resource Allocation**: The framework guides where to invest marketing and customer service resources, focusing on customers with high churn probability and a positive EV for intervention.
*   **Continuous Improvement**: The business parameters (Cost of Churn, Cost of Intervention, Probability of Success) should be regularly reviewed and updated based on new data, A/B test results, and market conditions. This allows the model to adapt and improve its recommendations over time.
*   **Feedback Loop**: Track the actual outcomes of interventions. Did customers who received a discount churn less? Was the predicted success rate accurate? Use this data to refine the `PROB_SUCCESS` parameters.

By adopting this data-driven approach, the company can move from reactive customer retention to a proactive, financially optimized strategy, significantly impacting the bottom line.

## Business Recommendations from Expected Value Analysis

Based on the Expected Value (EV) framework and the analysis of various intervention strategies, here are the key findings and actionable business recommendations:

### 1. Analysis of Total Expected Value

Let's revisit the total expected values for each strategy:

*   **No Intervention**: `$-452,433.74`
    *   This represents the total potential loss if no retention efforts are made for any customer. It's a significant negative value, highlighting the high cost of churn.

*   **Low-Cost Intervention**: `$30,230.12`
    *   Implementing a low-cost intervention (e.g., a 10% discount) for all customers with a churn probability above a certain threshold (if applied universally) results in a positive expected value, indicating a net financial gain compared to no intervention.

*   **High-Cost Intervention**: `$60,460.24`
    *   Implementing a high-cost intervention (e.g., a 20% discount) for all customers with a churn probability above a certain threshold (if applied universally) yields an even higher positive expected value, suggesting that for certain customers, a more significant discount is more profitable.

*   **Optimal Combined Strategy**: `$87,502.83`
    *   This is the most profitable strategy. By tailoring the intervention (No Intervention, Low-Cost, or High-Cost) to each customer based on their individual churn probability and the expected value of each action, the total expected value is maximized.

### 2. Distribution of Optimal Strategies

The `Distribution of Optimal Intervention Strategies` plot shows how many customers fall into each recommended category:

*   A large portion of customers might still be recommended for 'No Intervention', either because their churn probability is low, or the cost of intervention outweighs the potential benefit.
*   A significant number of customers are assigned to 'EV_Low_Intervention' and 'EV_High_Intervention', indicating that targeted discounts are financially beneficial for these segments.

### 3. Business Recommendation: Implement an Adaptive, Optimal Combined Strategy

**Recommendation**: The company should implement a dynamic, customer-specific retention strategy based on the 'Optimal Combined Strategy' identified by the Expected Value framework. This involves:

1.  **Predicting Churn Probability**: For each customer, use the `selected_best_model` (Random Forest, in this case) to predict their churn probability.
2.  **Calculating Expected Value for Interventions**: For each customer, calculate the Expected Value of 'No Intervention', 'Low-Cost Intervention', and 'High-Cost Intervention' using the defined business parameters.
3.  **Selecting the Optimal Action**: For each customer, choose the intervention strategy that yields the highest Expected Value.

### 4. Financial Implications

Implementing the **Optimal Combined Strategy** is projected to generate an additional expected value of approximately **`$87,502.83`**. This is a substantial improvement compared to the `$-452,433.74` expected loss if **No Intervention** is undertaken, representing a potential swing of over **`$500,000`**.

This framework ensures that resources (discounts) are allocated efficiently, targeting customers where the intervention is most likely to yield a positive financial return, and avoiding costly interventions for customers unlikely to churn or unlikely to be retained.

### 5. Optimizing Customer Retention Efforts

*   **Targeted Marketing**: Instead of generic marketing campaigns, retention efforts can be highly targeted. Customers identified for 'High-Cost Intervention' can receive personalized offers, while those for 'Low-Cost Intervention' can get more standard discounts.
*   **Resource Allocation**: The framework guides where to invest marketing and customer service resources, focusing on customers with high churn probability and a positive EV for intervention.
*   **Continuous Improvement**: The business parameters (Cost of Churn, Cost of Intervention, Probability of Success) should be regularly reviewed and updated based on new data, A/B test results, and market conditions. This allows the model to adapt and improve its recommendations over time.
*   **Feedback Loop**: Track the actual outcomes of interventions. Did customers who received a discount churn less? Was the predicted success rate accurate? Use this data to refine the `PROB_SUCCESS` parameters.

By adopting this data-driven approach, the company can move from reactive customer retention to a proactive, financially optimized strategy, significantly impacting the bottom line.


## Business Recommendations from Expected Value Analysis

Based on the Expected Value (EV) framework and the analysis of various intervention strategies, here are the key findings and actionable business recommendations:

### 1. Analysis of Total Expected Value

Let's revisit the total expected values for each strategy:

*   **No Intervention**: `$-452,433.74`
    *   This represents the total potential loss if no retention efforts are made for any customer. It's a significant negative value, highlighting the high cost of churn.

*   **Low-Cost Intervention**: `$30,230.12`
    *   Implementing a low-cost intervention (e.g., a 10% discount) for all customers with a churn probability above a certain threshold (if applied universally) results in a positive expected value, indicating a net financial gain compared to no intervention.

*   **High-Cost Intervention**: `$60,460.24`
    *   Implementing a high-cost intervention (e.g., a 20% discount) for all customers with a churn probability above a certain threshold (if applied universally) yields an even higher positive expected value, suggesting that for certain customers, a more significant discount is more profitable.

*   **Optimal Combined Strategy**: `$87,502.83`
    *   This is the most profitable strategy. By tailoring the intervention (No Intervention, Low-Cost, or High-Cost) to each customer based on their individual churn probability and the expected value of each action, the total expected value is maximized.

### 2. Distribution of Optimal Strategies

The `Distribution of Optimal Intervention Strategies` plot shows how many customers fall into each recommended category:

*   A large portion of customers might still be recommended for 'No Intervention', either because their churn probability is low, or the cost of intervention outweighs the potential benefit.
*   A significant number of customers are assigned to 'EV_Low_Intervention' and 'EV_High_Intervention', indicating that targeted discounts are financially beneficial for these segments.

### 3. Business Recommendation: Implement an Adaptive, Optimal Combined Strategy

**Recommendation**: The company should implement a dynamic, customer-specific retention strategy based on the 'Optimal Combined Strategy' identified by the Expected Value framework. This involves:

1.  **Predicting Churn Probability**: For each customer, use the `selected_best_model` (Random Forest, in this case) to predict their churn probability.
2.  **Calculating Expected Value for Interventions**: For each customer, calculate the Expected Value of 'No Intervention', 'Low-Cost Intervention', and 'High-Cost Intervention' using the defined business parameters.
3.  **Selecting the Optimal Action**: For each customer, choose the intervention strategy that yields the highest Expected Value.

### 4. Financial Implications

Implementing the **Optimal Combined Strategy** is projected to generate an additional expected value of approximately **`$87,502.83`**. This is a substantial improvement compared to the `$-452,433.74` expected loss if **No Intervention** is undertaken, representing a potential swing of over **`$500,000`**.

This framework ensures that resources (discounts) are allocated efficiently, targeting customers where the intervention is most likely to yield a positive financial return, and avoiding costly interventions for customers unlikely to churn or unlikely to be retained.

### 5. Optimizing Customer Retention Efforts

*   **Targeted Marketing**: Instead of generic marketing campaigns, retention efforts can be highly targeted. Customers identified for 'High-Cost Intervention' can receive personalized offers, while those for 'Low-Cost Intervention' can get more standard discounts.
*   **Resource Allocation**: The framework guides where to invest marketing and customer service resources, focusing on customers with high churn probability and a positive EV for intervention.
*   **Continuous Improvement**: The business parameters (Cost of Churn, Cost of Intervention, Probability of Success) should be regularly reviewed and updated based on new data, A/B test results, and market conditions. This allows the model to adapt and improve its recommendations over time.
*   **Feedback Loop**: Track the actual outcomes of interventions. Did customers who received a discount churn less? Was the predicted success rate accurate? Use this data to refine the `PROB_SUCCESS` parameters.

By adopting this data-driven approach, the company can move from reactive customer retention to a proactive, financially optimized strategy, significantly impacting the bottom line.


## Business Recommendations from Expected Value Analysis

Based on the Expected Value (EV) framework and the analysis of various intervention strategies, here are the key findings and actionable business recommendations:

### 1. Analysis of Total Expected Value

Let's revisit the total expected values for each strategy:

*   **No Intervention**: `$-452,433.74`
    *   This represents the total potential loss if no retention efforts are made for any customer. It's a significant negative value, highlighting the high cost of churn.

*   **Low-Cost Intervention**: `$30,230.12`
    *   Implementing a low-cost intervention (e.g., a 10% discount) for all customers with a churn probability above a certain threshold (if applied universally) results in a positive expected value, indicating a net financial gain compared to no intervention.

*   **High-Cost Intervention**: `$60,460.24`
    *   Implementing a high-cost intervention (e.g., a 20% discount) for all customers with a churn probability above a certain threshold (if applied universally) yields an even higher positive expected value, suggesting that for certain customers, a more significant discount is more profitable.

*   **Optimal Combined Strategy**: `$87,502.83`
    *   This is the most profitable strategy. By tailoring the intervention (No Intervention, Low-Cost, or High-Cost) to each customer based on their individual churn probability and the expected value of each action, the total expected value is maximized.

### 2. Distribution of Optimal Strategies

The `Distribution of Optimal Intervention Strategies` plot shows how many customers fall into each recommended category:

*   A large portion of customers might still be recommended for 'No Intervention', either because their churn probability is low, or the cost of intervention outweighs the potential benefit.
*   A significant number of customers are assigned to 'EV_Low_Intervention' and 'EV_High_Intervention', indicating that targeted discounts are financially beneficial for these segments.

### 3. Business Recommendation: Implement an Adaptive, Optimal Combined Strategy

**Recommendation**: The company should implement a dynamic, customer-specific retention strategy based on the 'Optimal Combined Strategy' identified by the Expected Value framework. This involves:

1.  **Predicting Churn Probability**: For each customer, use the `selected_best_model` (Random Forest, in this case) to predict their churn probability.
2.  **Calculating Expected Value for Interventions**: For each customer, calculate the Expected Value of 'No Intervention', 'Low-Cost Intervention', and 'High-Cost Intervention' using the defined business parameters.
3.  **Selecting the Optimal Action**: For each customer, choose the intervention strategy that yields the highest Expected Value.

### 4. Financial Implications

Implementing the **Optimal Combined Strategy** is projected to generate an additional expected value of approximately **`$87,502.83`**. This is a substantial improvement compared to the `$-452,433.74` expected loss if **No Intervention** is undertaken, representing a potential swing of over **`$500,000`**.

This framework ensures that resources (discounts) are allocated efficiently, targeting customers where the intervention is most likely to yield a positive financial return, and avoiding costly interventions for customers unlikely to churn or unlikely to be retained.

### 5. Optimizing Customer Retention Efforts

*   **Targeted Marketing**: Instead of generic marketing campaigns, retention efforts can be highly targeted. Customers identified for 'High-Cost Intervention' can receive personalized offers, while those for 'Low-Cost Intervention' can get more standard discounts.
*   **Resource Allocation**: The framework guides where to invest marketing and customer service resources, focusing on customers with high churn probability and a positive EV for intervention.
*   **Continuous Improvement**: The business parameters (Cost of Churn, Cost of Intervention, Probability of Success) should be regularly reviewed and updated based on new data, A/B test results, and market conditions. This allows the model to adapt and improve its recommendations over time.
*   **Feedback Loop**: Track the actual outcomes of interventions. Did customers who received a discount churn less? Was the predicted success rate accurate? Use this data to refine the `PROB_SUCCESS` parameters.

By adopting this data-driven approach, the company can move from reactive customer retention to a proactive, financially optimized strategy, significantly impacting the bottom line.


## Business Recommendations from Expected Value Analysis

Based on the Expected Value (EV) framework and the analysis of various intervention strategies, here are the key findings and actionable business recommendations:

### 1. Analysis of Total Expected Value

Let's revisit the total expected values for each strategy:

*   **No Intervention**: `$-452,433.74`
    *   This represents the total potential loss if no retention efforts are made for any customer. It's a significant negative value, highlighting the high cost of churn.

*   **Low-Cost Intervention**: `$30,230.12`
    *   Implementing a low-cost intervention (e.g., a 10% discount) for all customers with a churn probability above a certain threshold (if applied universally) results in a positive expected value, indicating a net financial gain compared to no intervention.

*   **High-Cost Intervention**: `$60,460.24`
    *   Implementing a high-cost intervention (e.g., a 20% discount) for all customers with a churn probability above a certain threshold (if applied universally) yields an even higher positive expected value, suggesting that for certain customers, a more significant discount is more profitable.

*   **Optimal Combined Strategy**: `$87,502.83`
    *   This is the most profitable strategy. By tailoring the intervention (No Intervention, Low-Cost, or High-Cost) to each customer based on their individual churn probability and the expected value of each action, the total expected value is maximized.

### 2. Distribution of Optimal Strategies

The `Distribution of Optimal Intervention Strategies` plot shows how many customers fall into each recommended category:

*   A large portion of customers might still be recommended for 'No Intervention', either because their churn probability is low, or the cost of intervention outweighs the potential benefit.
*   A significant number of customers are assigned to 'EV_Low_Intervention' and 'EV_High_Intervention', indicating that targeted discounts are financially beneficial for these segments.

### 3. Business Recommendation: Implement an Adaptive, Optimal Combined Strategy

**Recommendation**: The company should implement a dynamic, customer-specific retention strategy based on the 'Optimal Combined Strategy' identified by the Expected Value framework. This involves:

1.  **Predicting Churn Probability**: For each customer, use the `selected_best_model` (Random Forest, in this case) to predict their churn probability.
2.  **Calculating Expected Value for Interventions**: For each customer, calculate the Expected Value of 'No Intervention', 'Low-Cost Intervention', and 'High-Cost Intervention' using the defined business parameters.
3.  **Selecting the Optimal Action**: For each customer, choose the intervention strategy that yields the highest Expected Value.

### 4. Financial Implications

Implementing the **Optimal Combined Strategy** is projected to generate an additional expected value of approximately **`$87,502.83`**. This is a substantial improvement compared to the `$-452,433.74` expected loss if **No Intervention** is undertaken, representing a potential swing of over **`$500,000`**.

This framework ensures that resources (discounts) are allocated efficiently, targeting customers where the intervention is most likely to yield a positive financial return, and avoiding costly interventions for customers unlikely to churn or unlikely to be retained.

### 5. Optimizing Customer Retention Efforts

*   **Targeted Marketing**: Instead of generic marketing campaigns, retention efforts can be highly targeted. Customers identified for 'High-Cost Intervention' can receive personalized offers, while those for 'Low-Cost Intervention' can get more standard discounts.
*   **Resource Allocation**: The framework guides where to invest marketing and customer service resources, focusing on customers with high churn probability and a positive EV for intervention.
*   **Continuous Improvement**: The business parameters (Cost of Churn, Cost of Intervention, Probability of Success) should be regularly reviewed and updated based on new data, A/B test results, and market conditions. This allows the model to adapt and improve its recommendations over time.
*   **Feedback Loop**: Track the actual outcomes of interventions. Did customers who received a discount churn less? Was the predicted success rate accurate? Use this data to refine the `PROB_SUCCESS` parameters.

By adopting this data-driven approach, the company can move from reactive customer retention to a proactive, financially optimized strategy, significantly impacting the bottom line.


## Summary:

### Q&A
The most profitable discount strategy recommended is the "Optimal Combined Strategy," which is projected to generate a total expected value of \$87,502.83. This strategy involves tailoring interventions (No Intervention, Low-Cost, or High-Cost) to each customer based on their individual churn probability and the expected value of each action, rather than applying a single strategy across the board.

### Data Analysis Key Findings
*   Churn probabilities were successfully calculated for the test set using the `selected_best_model`.
*   Illustrative business parameters were defined for the Expected Value (EV) framework: Cost of Churn (\$500), Cost of High-Value Intervention (\$100), Cost of Low-Value Intervention (\$50), Probability of Success for High Intervention (0.60), and Probability of Success for Low Intervention (0.30).
*   The total Expected Value for each strategy across all customers was calculated as:
    *   No Intervention: $-\$452,433.74$
    *   Low-Cost Intervention: $\$30,230.12$
    *   High-Cost Intervention: $\$60,460.24$
    *   Optimal Combined Strategy: $\$87,502.83$
*   The "Optimal Combined Strategy" yields the highest total expected value, demonstrating a significant financial benefit of \$87,502.83 by personalizing retention efforts.
*   Implementing the "Optimal Combined Strategy" represents a potential financial swing of over \$500,000 compared to taking no retention action (from a loss of \$452,433.74 to a gain of \$87,502.83).

### Insights or Next Steps
*   Implement a dynamic, customer-specific retention strategy by predicting churn probability for each customer, calculating the Expected Value of various interventions, and selecting the optimal action that yields the highest EV.
*   Continuously review and update the business parameters (Cost of Churn, Cost of Intervention, Probability of Success) based on new data, A/B testing, and market conditions to refine and improve the model's recommendations over time.
